In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/ML and Image/Sheet_music_reader_Project/Mask_RCNN')

In [0]:
# fit a mask rcnn on the kangaroo dataset
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import tensorflow
print(tensorflow.__version__)

1.15.0


Using TensorFlow backend.


In [0]:
# class that defines and loads the kangaroo dataset
class MusicDataset(Dataset):
	# load the dataset definitions
	def load_dataset(self, dataset_dir, is_train=True):
		# define one class
		self.add_class("dataset", 1, "treble")
		self.add_class("dataset", 2, "bass")
		self.add_class("dataset", 3, "black note")
		self.add_class("dataset", 4, "white note")
		self.add_class("dataset", 5, "whole note")

		# define data locations
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annots/'
		# find all images
		for filename in listdir(images_dir):
			# extract image id
			image_id = filename[:-4]
			# skip bad images
			# if image_id in ['00261']:
			# 	continue
			# skip all images after 150 if we are building the train set
			if is_train and int(image_id) >= 177 and int(image_id) <= 196:
				print('test : ', int(image_id))
				continue
			# skip all images before 150 if we are building the test/val set
			if not is_train and int(image_id) < 177 and int(image_id) > 196:
				print('train : ', int(image_id))
				continue
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			# add to dataset
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)
 
	# extract bounding boxes from an annotation file
	def extract_boxes(self, filename, className = 'all'):
		# load and parse the file
		tree = ElementTree.parse(filename)
		# print('tree : {}' .format(tree))
		# get the root of the document
		root = tree.getroot()

		boxes = list()
		# extract each bounding box
		for Object in root.findall('object'):
			Name = Object.find('name')
			# print(Name.text)
			#	can change interest box from this
			if Name.text != className and className != 'all':
				continue
			box = Object.find('bndbox')
			xmin = int(box.find('xmin').text)
			ymin = int(box.find('ymin').text)
			xmax = int(box.find('xmax').text)
			ymax = int(box.find('ymax').text)
			#	add box and class name in list
			coors = [xmin, ymin, xmax, ymax, Name.text]
			boxes.append(coors)
		
		# extract image dimensions
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height
 
	# load the masks for an image
	def load_mask(self, image_id, className = 'all'):
		# get details of image
		info = self.image_info[image_id]
		# print('info : {}' .format(info))
		# define box file location
		path = info['annotation']
		# load XML
		boxes, w, h = self.extract_boxes(path, className)
		# create one array for all masks, each on a different channel
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		# create masks
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			masks[row_s:row_e, col_s:col_e, i] = 1
			# if image_id == 4:
			# 	print('class_names : {}' .format(box[4]))
			class_ids.append(self.class_names.index(box[4]))
		return masks, asarray(class_ids, dtype='int32')
 
	# load an image reference
	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

In [0]:
# define a configuration for the model
class DatasetsConfig(Config):
	# define the name of the configuration
	NAME = "musicdatasets_cfg"
	# number of classes (background + kangaroo)
	NUM_CLASSES = 1 + 5
	# number of training steps per epoch
	STEPS_PER_EPOCH = 131

In [0]:
# prepare train set
train_set = MusicDataset()
train_set.load_dataset('datasets', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
test_set = MusicDataset()
test_set.load_dataset('datasets', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
# prepare config
config = DatasetsConfig()
config.display()
# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)
# load weights (mscoco) and exclude the output layers
model.load_weights('mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
# train weights (output layers or 'heads')
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=100, layers='heads')

test :  177
test :  178
test :  179
test :  180
test :  181
test :  182
test :  183
test :  184
test :  185
test :  186
test :  187
test :  188
test :  189
test :  190
test :  191
test :  192
test :  193
test :  194
test :  195
test :  196
Train: 241
Test: 261

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                18
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMEN

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/100
131/131 [==============================] - 554s 4s/step - loss: 2.0676 - rpn_class_loss: 0.1590 - rpn_bbox_loss: 0.5855 - mrcnn_class_loss: 0.4657 - mrcnn_bbox_loss: 0.4196 - mrcnn_mask_loss: 0.4377 - val_loss: 1.4850 - val_rpn_class_loss: 0.0756 - val_rpn_bbox_loss: 0.4526 - val_mrcnn_class_loss: 0.2960 - val_mrcnn_bbox_loss: 0.3255 - val_mrcnn_mask_loss: 0.3353

Epoch 2/100
131/131 [==============================] - 231s 2s/step - loss: 1.2964 - rpn_class_loss: 0.0614 - rpn_bbox_loss: 0.4144 - mrcnn_class_loss: 0.2321 - mrcnn_bbox_loss: 0.2731 - mrcnn_mask_loss: 0.3154 - val_loss: 1.3365 - val_rpn_class_loss: 0.0505 - val_rpn_bbox_loss: 0.4003 - val_mrcnn_class_loss: 0.2420 - val_mrcnn_bbox_loss: 0.3414 - val_mrcnn_mask_loss: 0.3022
Epoch 3/100
131/131 [==============================] - 400s 3s/step - loss: 1.1373 - rpn_class_loss: 0.0630 - rpn_bbox_loss: 0.3485 - mrcnn_class_loss: 0.2115 - mrcnn_bbox_loss: 0.2223 - mrcnn_mask_loss: 0.2920 - val_loss: 1.0800 - val_rpn_c

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:709: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


131/131 [==============================] - 396s 3s/step - loss: 0.7306 - rpn_class_loss: 0.0419 - rpn_bbox_loss: 0.2438 - mrcnn_class_loss: 0.1232 - mrcnn_bbox_loss: 0.1083 - mrcnn_mask_loss: 0.2133 - val_loss: 0.6790 - val_rpn_class_loss: 0.0371 - val_rpn_bbox_loss: 0.1972 - val_mrcnn_class_loss: 0.1287 - val_mrcnn_bbox_loss: 0.1115 - val_mrcnn_mask_loss: 0.2044
Epoch 12/100
131/131 [==============================] - 365s 3s/step - loss: 0.6781 - rpn_class_loss: 0.0374 - rpn_bbox_loss: 0.2186 - mrcnn_class_loss: 0.1207 - mrcnn_bbox_loss: 0.0987 - mrcnn_mask_loss: 0.2027 - val_loss: 0.6620 - val_rpn_class_loss: 0.0347 - val_rpn_bbox_loss: 0.1907 - val_mrcnn_class_loss: 0.1238 - val_mrcnn_bbox_loss: 0.1117 - val_mrcnn_mask_loss: 0.2011
Epoch 13/100
131/131 [==============================] - 403s 3s/step - loss: 0.6971 - rpn_class_loss: 0.0391 - rpn_bbox_loss: 0.2298 - mrcnn_class_loss: 0.1199 - mrcnn_bbox_loss: 0.1013 - mrcnn_mask_loss: 0.2070 - val_loss: 0.7073 - val_rpn_class_loss: 0.